# NLP를 활용한 레시피 추천 챗봇 구현
**개요**
- Collection 
- Preprocessing
- EDA
- Embeddings
- Modeling
- Streamlit

## 5. Modeling
**모델링 과정**
1. 사전 학습된 모델을 Sentence Transformer 를 통해 불러오기
2. 수집하고 기본적인 전처리를 거친 데이터를 위의 모델을 통해 임베딩 벡터로 변환한 파생변수 생성하기
3. input 으로 사용자가 재료를 포함한 문자열을 입력하면 문장을 벡터화하여 기존의 임베딩 벡터와 코사인 유사도를 구하는 함수
4. 코사인 유사도 상위 기준으로 n개를 output 으로 추천
5. OpenAI 의 API 를 활용하여 사용자의 문장 형태 input 의 의도를 파악하여 모델링 함수를 실행하도록 연동
6. streamlit 에 연동하여 배포


### 라이브러리 import

In [1]:
# 데이터 분석
import pandas as pd
import numpy as np

# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 진행시간 표시
import swifter
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
# 파이토치
import torch

# 문장 임베딩, transformer 유틸리티
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import SentenceTransformer, models

In [3]:
# 객체 복사
import copy

# JSON 형식 데이터 처리
import json

# 데이터 수집
import requests

In [4]:
# 데이터베이스 활용
import sqlite3 
import pickle

In [5]:
# OpenAI API 활용
import openai 
import os # 운영체제
import sys # 파이썬 변수, 함수 엑세스 
from dotenv import load_dotenv # 환경 변수 로드(API Key 보안)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [6]:
# 실행 os 확인
cur_os = sys.platform
cur_os

'darwin'

### 데이터 불러오기

In [7]:
df = pd.read_pickle('data/compact_kosroberta_recipes.pkl')
df.shape

(4340, 12)

### 파생 변수
- feature1 = '재료'
- feature2 = '재료' + '요리'
- feature3 = '재료' + '요리' + '종류'
- feature4 = '재료' + '요리' + '종류' + '난이도'
- feature5 = '재료' + '요리' + '종류' + '난이도' + '요리방법'
- **feature = '재료' + '요리' + '설명' + '종류' + '난이도' + '요리방법'**

### 모델 불러오기
- model : Sentence Transformer
- pre trained model : `jhgan/ko-sroberta-multitask`

In [8]:
model_name = 'jhgan/ko-sroberta-multitask'

In [9]:
model = SentenceTransformer(model_name)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [10]:
df['feature'][0]

"통깨 꼬시래기 식초 용냉면육수 매실청 쌈무 참기름 고춧가루 소금 오이 고추장 달걀꼬시래기 물냉면꼬시래기는 해조류의 한 종류인데요. 지방, 탄수화물 함량이 낮고 칼슘과 식이섬유를 풍부하게 함유하고 있는 건강 식품이에요. 쫄깃쫄깃하고 꼬들꼬들한 식감이 특징이라 면 대신 사용하기 좋습니다. 특히 냉면으로 만들어 먹으면 제격인데요. 간단한 재료들만 있으면 건강하면서도 시원한 꼬시래기 물냉면을 즐길 수 있답니다!메인요리쉬움['끓는 물에 소금, 식초, 꼬시래기를 넣어 30초 정도 데쳐주세요.\\r\\n(tip. 소금과 식초를 넣어 데치면 비린맛이 제거돼요)', '체에 받쳐 찬물에 충분히 헹궈 물기를 빼주세요. ', '오이는 채썰고 쌈무도 먹기 좋은 크기로 잘라주세요. 달걀은 반으로 잘라주세요.', '볼에 양념 재료를 넣어 섞어주세요. ', '그릇에 꼬시래기, 쌈무, 오이, 양념장, 달걀을 올려주세요. 통깨를 뿌린 후 달걀을 얹어주세요. 냉면 육수를 부어 완성해주세요. ']"

## 콘텐츠 추천 함수(코사인 유사도 기반)

In [11]:
def get_similar_recipes(query, model, df, exclude_ingredients=None, intent=None):
    data = {'요리': df['요리'], 'feature': df['ko-sroberta-multitask-feature']}
    query_encode = model.encode(query)
    cos_scores = util.pytorch_cos_sim(query_encode, data['feature'])[0]
    top_results = torch.topk(cos_scores, k=100)

    # top_results에서 exclude 조건 적용
    if exclude_ingredients:
        if len(exclude_ingredients) > 1:
            exclude_mask = np.array([any(exclude in row for exclude in exclude_ingredients) for row in df['재료']])
        else:
            exclude_mask = np.array([exclude_ingredients[0] in row for row in df['재료']])
        exclude_idx = np.where(exclude_mask)[0]
        exclude_idx_tensor = torch.tensor(exclude_idx)
        top_results = (top_results[0][~torch.isin(top_results[1], exclude_idx_tensor)],
                        top_results[1][~torch.isin(top_results[1], exclude_idx_tensor)])
    
    if intent == '1':
        result_df = df.iloc[top_results[1].numpy(), :][['요리', '종류', '재료', '설명', '난이도', '링크']]
        result_df = result_df.drop_duplicates(subset=['링크']).head(3)
    elif intent == '2':
        result_df = df.iloc[top_results[1].numpy(), :][['요리', '종류', '재료', '설명', '난이도', '링크']]
        result_df = result_df.drop_duplicates(subset=['링크']).sample(3)
    else:
        result_df = df.iloc[top_results[1].numpy(), :][['요리', '종류', '재료', '설명', '난이도', '링크']]
        result_df = result_df.drop_duplicates(subset=['링크']).head(5)
    
    return result_df

In [12]:
# 일반 추천
include = '여름 메뉴 추천해줘'
intent = '1'
exclude = []
result = get_similar_recipes(include, model, df, exclude_ingredients=exclude, intent=intent)
result

/Users/choimingue/anaconda3/envs/final/lib/python3.10/site-packages/sentence_transformers/util.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1678454852765/work/torch/csrc/utils/tensor_new.cpp:248.)
  b = torch.tensor(b)


,요리,종류,재료,설명,난이도,링크
3074,파인애플 오이 샐러드,양식,후춧가루 파인애플 양파 메이플시럽 고수 소금 라임 라임제스트 오이 올리브오일 라임즙,Welcome! 우식 와인 Bar!\r\n 말만 들어도 시원해지는 샐러드 레시피!...,쉬움,https://wtable.co.kr/recipes/Mb7rr7JyP9CnKAiin...
1804,오이 냉수프,간단요리,후춧가루 얼음 소금 오이 레몬 올리브오일,오이는 수분이 많고 찬 성질을 가지고 있어 몸의 열을 내리고 갈증을 풀어줍니다. 오...,쉬움,https://wtable.co.kr/recipes/JhTGA1Wrv8djRykzL...
2820,오이지무침,한식,통깨 참기름 다진마늘 설탕 홍고추 물엿 고춧가루 쪽파 오이지,여름 제철 재료인 오이! 수분이 많고 아삭거리는 식감 덕분에 오이를 좋아하는 분들...,쉬움,https://wtable.co.kr/recipes/otoQ5JpRjgNPpuDts...


In [13]:
# 재추천
include = '여름 메뉴 추천해줘'
intent = '2'
exclude = []
result = get_similar_recipes(include, model, df, exclude_ingredients=exclude, intent=intent)
result

,요리,종류,재료,설명,난이도,링크
12,[우정욱 요리연구가] 봄 채소 영양솥밥,메인요리,현미튀김 냉동유부 우엉 설탕 흑곤약 쌀 냉이 깻잎순 소금 생표고 물 양조간장 미림 ...,별다른 반찬 없이 한 그릇만으로도 만족스러운 식사를 선물하는 솥밥이 요즘의 트렌드잖...,보통,https://wtable.co.kr/recipes/iKnuu9Ro34oKaCNdr...
1260,과일아이스바,초대요리,올리고당 아이스바몰드 블루베리 민트잎 청포도주스 천도복숭아 체리 키위,"집에서 만든 시원하고 건강한 간식, 과일 아이스바예요. 별다른 장식 없이도 콕콕 박...",쉬움,https://wtable.co.kr/recipes/3zTes1ZpAXi6MwmTL...
1873,가지 냉국,간단요리,양조식초 진간장 설탕 홍고추 양파 가지 물,여름 제철 채소 가지의 특별한 변신. 뜨거운 여름날에도 뼛속까지 시원하게 즐기는 입...,쉬움,https://wtable.co.kr/recipes/1yBzxrws5xiTzxVxU...


In [14]:
# 단일 제외
include = '여름 메뉴 추천해줘'
intent = '1'
exclude = ['오이']
result = get_similar_recipes(include, model, df, exclude_ingredients=exclude, intent=intent)
result

,요리,종류,재료,설명,난이도,링크
1494,토마토빙수,간식,연유 스노우요거트얼음 설탕 완숙토마토 다진피스타치오 플레인요거트 바질잎 토마토퓌레 우유,여름아 안녕! 여름이 점점 다가오고 있어요. 여름에 아이들이 가장 먼저 찾는 음식...,쉬움,https://wtable.co.kr/recipes/ZYJAE5LVy1wCv36Zi...
4109,템페 구이 & 퀴노아 샐러드 밀프렙,키토,템페 후추 식초 메이플시럽 소금 간장 올리브오일 파프리카파우더,바쁜 일상 중 단백질이 가득한 식단을 챙기기란 쉽지 않죠? 그렇다면 미리 준비해놓고...,보통,https://wtable.co.kr/recipes/padCkSrxLSYaY61oc...
1487,복숭아 병조림,간식,설탕 물 레몬즙 베이킹소다 단단한복숭아,추억이 가득한 여름을 그냥 보낼 수는 없죠. 여름을 추억하는 방법! 여름 과일 복...,쉬움,https://wtable.co.kr/recipes/P9787UdUdzMYgUovT...


In [15]:
# 다중 제외
include = '여름 메뉴 추천해줘'
intent = '1'
exclude = ['오이', '토마토']
result = get_similar_recipes(include, model, df, exclude_ingredients=exclude, intent=intent)
result

,요리,종류,재료,설명,난이도,링크
4109,템페 구이 & 퀴노아 샐러드 밀프렙,키토,템페 후추 식초 메이플시럽 소금 간장 올리브오일 파프리카파우더,바쁜 일상 중 단백질이 가득한 식단을 챙기기란 쉽지 않죠? 그렇다면 미리 준비해놓고...,보통,https://wtable.co.kr/recipes/padCkSrxLSYaY61oc...
1487,복숭아 병조림,간식,설탕 물 레몬즙 베이킹소다 단단한복숭아,추억이 가득한 여름을 그냥 보낼 수는 없죠. 여름을 추억하는 방법! 여름 과일 복...,쉬움,https://wtable.co.kr/recipes/P9787UdUdzMYgUovT...
885,새우튀김냉메밀,메인요리,실파 다시마 소금 맛술 물 간장 무순 김 식용유 무 가쓰오부시 고추냉이 튀김가루 설...,여름철 더위로 입맛이 없을 때는 시원한 국수 한 그릇만한 게 없지요. 특히 메밀은 ...,보통,https://wtable.co.kr/recipes/LjiJ7qCnJ3V4ZxdnW...


## ChatGPT 연동하기
### 세 가지 Role
- user : 마치 채팅하는 것처럼 ChatGPT에 직접 무언가를 물어보고 싶을 때 사용하는 role (ex. "Please explain what AI is")
- system : 유저에게 메시지를 받기 전에 모델을 초기화하거나 구성하려는 경우 사용하는 role (ex. "You are a helpful kindergarten teacher talking to children")
- assistant : 이전에 ChatGPT가 유저에게 보낸 메시지가 무엇인지 알려주는 role. 유저와 어시스턴트 사이의 대화를 저장하고 어시스턴트에게 이전 대화를 전달하여 응답값을 조정할 수 있음. (ex. 이전까지의 대화를 기억하게 함으로써 명사 -> 대명사로 이어지는 맥락을 이해할 수 있도록 해줌)
### Fine tuning
Role 을 지정하는 것 이외에 Fine-tuning 을 활용하는 것이 실제 서비스에서는 더 이상적인 형태
사용자의 데이터를 가지고 특정 태스크에 알맞게 커스텀하는 것
- `{"prompt" : "I really like this recipe!", "completion" : "positive"}`
- `{"prompt" : "I'd like to leave out the cucumber.", "completion" : "negative"}`